## 1) 정확도(Accuracy)

어떤 문제점이 발생할 수 있는지?  
레이블 값이 7인 것만 True, 나머지 값은 False -> 이진 분류 문제로 바꿔 봄 <불균형한 데이터 세트로 변형>  
<br>
-> 데이터 분포도가 균일하지 않은 경우, 높은 수치가 나타날 수 있음을 보여주기 위함

### 1. 불균형한 데이터 세트와 Dummy Classifier를 생성함

In [1]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

In [2]:
class MyFakeClassifier(BaseEstimator):
    def fit(self, X, y):
        pass
    
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

In [3]:
digits = load_digits()

y = (digits.target == 7).astype(int)  # digits 번호가 7번이면, True이고 이를 astype(int)로 1로 변환
X_train, X_test, y_train, y_test = train_test_split(digits.data, y, random_state=11)

### 2. 불균형한 데이터로 생성한 y_test의 데이터 분포도 확인

In [4]:
print("레이블 테스트 세트 크기: ", y_test.shape)
print("테스트 세트 레이블 0과 1의 분포도")
print(pd.Series(y_test).value_counts())

레이블 테스트 세트 크기:  (450,)
테스트 세트 레이블 0과 1의 분포도
0    405
1     45
dtype: int64


### 3. 예측과 평가 수행

In [5]:
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train, y_train)
fakepred = fakeclf.predict(X_test)
print("모든 예측을 0으로 하여도 정확도는: {:.3f}".format(accuracy_score(y_test, fakepred)))

모든 예측을 0으로 하여도 정확도는: 0.900


## 2) 오차 행렬(confusion matrix)

In [6]:
from sklearn.metrics import confusion_matrix

- TN(예측을 Negative로, 실제도 Negative): array[0, 0] = 405  
- FP(예측을 Positive로, 실제는 Negative): array[0, 1] = 0  
- FN(예측을 Negative로, 실제는 Positive): array[1, 0] = 45  
- TP(예측을 Positive로, 실제도 Positive): array[1, 1] = 0  

In [7]:
confusion_matrix(y_test, fakepred)

array([[405,   0],
       [ 45,   0]])

## 3. 정밀도(Precision)와 재현율(Recall)

In [8]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

#### 데이터 가공 함수

In [21]:
from sklearn import preprocessing

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)  
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True)
    
    return df

# 머신러닝 알고리즘에 불필요한 속성 제거 함수
def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return df

# 레이블 인코딩 수행 함수
def format_features(df):
    df['Cabin'] = df['Cabin'].str[0]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = preprocessing.LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    
    return df

# 앞에서 설정한 데이터 전처리 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

#### confusion_matrix, accuracy, precision, recall 평가를 한번에 호출하는 get_clf_eval() 함수

In [9]:
def get_clf_eval(y_test, pred):
    accuracy = accuracy_score(y_test, pred)
    confusion = confusion_matrix(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    print("오차 행렬")
    print(confusion)
    print("정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}".format(accuracy, precision, recall))

#### 로지스틱 회귀 기반, 타이타닉 생존자를 예측하고 평가를 수행하는 코드

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

titanic_df = pd.read_csv('./titanic_train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)
X_titanic_df = transform_features(X_titanic_df)

X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, 
                                                    test_size=0.2, random_state=11)

lr_clf = LogisticRegression()

lr_clf.fit(X_train, y_train)
pred = lr_clf.predict(X_test)
get_clf_eval(y_test, pred)

오차 행렬
[[104  14]
 [ 13  48]]
정확도: 0.8492, 정밀도: 0.7742, 재현율: 0.7869


/Users/min/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
